### 配置文件验证

In [1]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

In [2]:
from modelModule.model2 import VAE2
import torch

In [3]:
model = VAE2(dim=57, nhead=3)
a = torch.randn((10,57))
b = torch.randn((10,57))
x, y, z = model(a,b)

In [4]:
x.shape

torch.Size([10, 57])

### dataloader验证

In [ ]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [ ]:
dataset = FlatDataset('Spam_train.csv')
print(dataset.Max_Val.shape)
print(dataset.Min_Val)

In [ ]:
dataset = FlatDataset('Letter_train.csv')
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [ ]:
test_data = imputed_data * global_max + global_min # 广播效应

In [ ]:
a = torch.randn(2,3,4)
b = torch.nn.AdaptiveMaxPool1d(output_size=1)(a).squeeze(-1)

In [ ]:
b.shape

### 模型精度测定

In [3]:
from dataModule.dataset1 import FlatDataset
# from modelModule.model1 import VAE
from modelModule.model2 import VAE2
from torch.utils.data import DataLoader
from utils import result_show, get_missing
import torch
import numpy as np

In [4]:
## 模型参数载入
model = VAE2(dim=57, nhead=3)
checkpoint_path = r'模型参数保存/Spam_model2_norm_way_mean_norm_/version_1/checkpoints/epoch=227-step=3420.ckpt'    
data_norm = 'mean_norm'  # 数据集采用正则化方式minmax_norm,mean_norm
train_csv = r'Spam_train.csv'
test_csv = r'Spam_test.csv'

checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
## 载入训练集的均值(最小)方差(最大)
train_dataset = FlatDataset(train_csv, data_norm=data_norm)
model.get_global_min_max(global_max=train_dataset.Max_Val, global_min=train_dataset.Min_Val)
print('载入成功')

载入成功


In [5]:
test_dataset = FlatDataset(test_csv, data_norm=data_norm)
data_size = 10
src_data = np.stack([test_dataset[index]['src_data'] for index in range(data_size)], axis=0)
normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(data_size)], axis=0)
_, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
input = src_data.copy()
input[miss_matrix == 0] = np.nan

In [6]:
imputed_data, output = model.inference(input, miss_matrix) # inference输出的结果已经重构后的结果
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()

In [7]:
result_show(src_data, imputed_data, miss_matrix)

原始数据 [8.000e-01 1.000e-01 1.000e-01 1.310e+00 2.000e-01 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 1.110e-01 0.000e+00 1.580e-01 1.351e+03
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 4.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 9.090e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 1.666e+00 1.500e+01 0.000e+00 0.000e+00 3.200e-01 1.600e-01 0.000e+00
 0.000e+00 0.000e+00 1.600e-01 0.000e+00 1.470e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 2.900e-02 1.300e+01 1.580e+02
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 

array([8.23394908e-01, 4.81192362e+00, 5.22506389e+00, 5.82205508e-01,
       1.39467399e+00, 4.27139010e+07, 2.04355821e+07, 3.69770532e+06,
       1.99405883e+07, 1.15498177e+07, 1.38867360e+00, 6.67813593e+06,
       5.88403460e-02, 7.76286447e-01, 2.19439923e+07, 2.71461623e+05,
       2.54411068e+07, 1.41380239e+07, 3.19675283e+07, 7.87745449e+06,
       1.65158842e+07, 1.60777770e+07, 2.02543643e+07, 2.43731076e+07,
       8.58226773e+06, 3.53835252e+07, 8.49495870e+06, 2.28380325e+06,
       6.64678183e+06, 5.26428446e+07, 1.47681481e+07, 7.44637446e+01,
       2.19200310e+07, 1.01543599e+08, 2.56626012e+05, 7.41768237e+06,
       1.04947717e+07, 2.06599771e+08, 1.63975478e+07, 7.35269364e+07,
       2.01937988e+07, 4.26408504e+07, 1.14322191e+07, 4.54437023e+06,
       1.03888384e+00, 4.51987762e+07, 2.97003479e+05, 1.13588851e+07,
       6.61996572e+06, 1.32499063e+00, 1.90950813e+01, 2.19578732e+07,
       3.50413745e+07, 5.58554318e-01, 1.46614059e+00, 1.05142059e+07,
      

In [10]:
imputed_data

array([[ 2.14650995e-01,  9.66677355e-01,  1.59321301e-02,
         3.07760979e-01,  2.61114240e-01,  1.37836326e-01,
         1.43129061e-01, -6.14348519e-02, -9.17590184e-02,
         5.44047840e-01,  9.21930866e-03,  6.62874640e-01,
        -5.63370100e-02, -7.81178994e-02,  1.34973695e-01,
         6.10303376e-01,  3.26908852e-01,  9.62150538e-02,
         2.05998535e+00, -6.67448819e-02,  6.33488838e-01,
         4.84912365e-01,  7.08946473e-02,  1.70145506e-01,
         7.71645127e-01,  1.70566600e-01,  8.15368326e-01,
        -1.78035260e-01,  2.42935520e-01, -1.47751521e-02,
        -1.10080786e-01,  1.26534516e-01,  3.97732235e-01,
        -7.25282804e-02,  9.48811227e-02,  2.01409670e-01,
         1.90410121e-01, -4.42566302e-02,  7.68581503e-02,
         3.73398258e-02,  1.89243126e-01, -3.19244922e-01,
        -3.33930650e-02,  4.19950704e-01,  5.30696313e-01,
         8.14977587e-02,  2.39612393e-02, -1.38583269e-03,
         1.17227848e-01, -3.66001428e-02,  6.56482465e-0

In [ ]:
imputed_data.shape

In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))